In [1]:
import cv2
from sensor_msgs.msg import Image
from cv_bridge import CvBridge
import rospy

import pyrealsense2
from realsense_depth import *
import numpy as np
from matplotlib import pyplot as plt

import open3d as o3d
from scipy import spatial
from scipy.spatial.transform import Rotation as R
import math
import tf2_ros, tf
import geometry_msgs.msg
import PyKDL
import time

import sys
import moveit_commander
import moveit_msgs.msg
from moveit_commander.conversions import pose_to_list

import subprocess
from ast import literal_eval
import copy
from PIL import Image as PIL_img
from io import BytesIO


image = None


rospy.init_node('my_pic', anonymous=True)

moveit_commander.roscpp_initialize(sys.argv)
robot = moveit_commander.RobotCommander()
scene = moveit_commander.PlanningSceneInterface()
group_name = "panda_arm"
move_group = moveit_commander.MoveGroupCommander(group_name) #we'll pass it on while calling functions


tfbuffer = tf2_ros.Buffer()
listener = tf2_ros.TransformListener(tfbuffer)
br = tf2_ros.TransformBroadcaster()
t = geometry_msgs.msg.TransformStamped()

static_br = tf2_ros.StaticTransformBroadcaster()
static_t = geometry_msgs.msg.TransformStamped()

bridge = CvBridge()
loop_rate = rospy.Rate(0.5) # Node cycle rate (in Hz).
np.set_printoptions(suppress=True)

# Initialize Camera Intel Realsense
#dc = DepthCamera()
print("loaded")

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.
loaded


In [19]:
def grab_frame():
   
    frame_color=rospy.wait_for_message('/camera/color/image_raw', Image, timeout=None) #wait_for_message(topic, topic_type, timeout=None): 
    cv_image_color = bridge.imgmsg_to_cv2(frame_color, desired_encoding='rgb8')
    #frame_depth = rospy.wait_for_message('/camera/aligned_depth_to_color/image_raw', Image, timeout=None) #wait_for_message(topic, topic_type, timeout=None): 
    frame_depth = rospy.wait_for_message('/camera/depth/image_rect_raw', Image, timeout=None) #wait_for_message(topic, topic_type, timeout=None): 
    cv_image_depth = bridge.imgmsg_to_cv2(frame_depth)
    
    return cv_image_color, cv_image_depth

def Generate_PointCloud(color_frame,depth_frame,from_depth=False):

    #intrinsic.intrinsic_matrix =  [[602.71783447, 0.0, 313.06835938], [0.0, 601.61364746, 230.37461853], [0.0, 0.0, 1.0]]
    '''
     Intrinsic of "Color" / 640x480 / {YUYV/RGB8/BGR8/RGBA8/BGRA8/Y8/Y16}
      Width:      	640
      Height:     	480
      PPX:        	313.068359375
      PPY:        	230.374618530273
      Fx:         	602.717834472656
      Fy:         	601.613647460938
      Distortion: 	Inverse Brown Conrady
      Coeffs:     	0  	0  	0  	0  	0  
      FOV (deg):  	55.93 x 43.49
    '''
    
    #From camera  Intrinsic of "Depth" / 640x480 / {Z16} 
    #Distortion: Brown Conrady  Coeffs:  0  0  0  0  0   FOV (deg): 79.93 x 64.3
    w = 640
    h = 480
    fx = 381.838836669922
    fy = 381.838836669922
    cx = 320.060424804688
    cy = 237.036407470703
    
    #Depth frame ROS camera
    #w = 640
    #h = 480
    #fx = 347.99755859375
    #fy = 347.99755859375
    #cx = 320.0
    #cy = 240.0    
    
    cam = o3d.camera.PinholeCameraParameters()
    cam.intrinsic = o3d.camera.PinholeCameraIntrinsic(w, h, fx,fy, cx, cy)
    #intrinsic.intrinsic_matrix = [[fx, 0, cx], [0, fy, cy], [0, 0, 1]]  #alternative method
    '''   
    extr_T = [0.0150703,0.000300758,9.12806e-06] #Depth to RGB
    extr_R = [[0.999992,0.00279048,0.00277953],
              [-0.00280054,0.99999,0.00362144],
              [-0.0027694,-0.00362919,0.99999]]  '''

    
    extr_T = [-0.0150693,-0.000342775,-5.21054e-05]   #RGB to Depth
    extr_R = [[0.999992,-0.00280054,-0.0027694],
              [0.00279048,0.99999,-0.00362919],
              [0.00277953,0.00362144,0.99999]]  
    
    cam.extrinsic = np.array([[extr_R[0][0], extr_R[0][1], extr_R[0][2], extr_T[0]], 
                                 [extr_R[1][0], extr_R[1][1], extr_R[1][2], extr_T[1]], 
                                 [extr_R[2][0], extr_R[2][1], extr_R[2][2], extr_T[2]], 
                                 [0.          , 0.          , 0.          , 1.      ]])

    color_raw = o3d.geometry.Image(np.asarray(color_frame))
    depth_raw = o3d.geometry.Image(np.asarray(depth_frame.astype(np.uint16)))
    
    if from_depth==False:
        rgbd_image = o3d.geometry.RGBDImage.create_from_color_and_depth(color_raw, depth_raw, convert_rgb_to_intensity=False)
        pcd = o3d.geometry.PointCloud.create_from_rgbd_image(rgbd_image, cam.intrinsic, cam.extrinsic)
        #pcd = o3d.geometry.PointCloud.create_from_rgbd_image(rgbd_image, cam.intrinsic)
    else:
        pcd =  o3d.geometry.PointCloud.create_from_depth_image(depth_raw, cam.intrinsic, cam.extrinsic, 
                                                               depth_scale=1000.0, depth_trunc=1000.0, 
                                                               stride=1, project_valid_depth_only=True)
    
    pcd.transform([[1, 0, 0, 0], [0, -1, 0, 0], [0, 0, -1, 0], [0, 0, 0, 1]]) # Flip it, otherwise the pointcloud will be upside down.
    return pcd


def fetch_transform(tfbuffer,frame1,frame2,quat=0):
    flag = 0
    while flag==0:
        try:
            trans = tfbuffer.lookup_transform(frame1, frame2, rospy.Time(),rospy.Duration(8.0))
            #print (trans)
            trans = trans.transform  #save translation and rotation
            #rot = PyKDL.Rotation.Quaternion(* [ trans.rotation.x,trans.rotation.y,trans.rotation.z,trans.rotation.w] )
            #ypr = [ i  / np.pi * 180 for i in rot.GetEulerZYX() ]
            #print(ypr[2],ypr[1],ypr[0])
            
            rot = R.from_quat([ trans.rotation.x,trans.rotation.y,trans.rotation.z,trans.rotation.w]) #creates rotation matrix
            rpy = rot.as_euler('XYZ',degrees=True) #extrinsic
            break
        except (tf2_ros.LookupException, tf2_ros.ConnectivityException, tf2_ros.ExtrapolationException) as e:
            #print ("Fail", e)
            continue
    if quat==0:
        return trans.translation.x, trans.translation.y, trans.translation.z, rpy[0],rpy[1],rpy[2] #ypr[2], ypr[1], ypr[0]
    elif quat==1:
        return trans.translation.x, trans.translation.y, trans.translation.z, trans.rotation.x, trans.rotation.y, trans.rotation.z, trans.rotation.w
    


def Axis_angle_to_Quat(vector,angle):
    x=vector[0]
    y=vector[1]
    z=vector[2]
    qx = x * np.sin(angle/2)
    qy = y * np.sin(angle/2)
    qz = z * np.sin(angle/2)
    qw = np.cos(angle/2)
    
    return qx,qy,qz,qw

def Axis_angle_to_Euler(vector,angle):
    x=vector[0]
    y=vector[1]
    z=vector[2]
    s=np.sin(angle)
    c=np.cos(angle)
    t=1-c
    
    if ((x*y*t + z*s) > 0.998):  #north pole singularity detected
        
        heading = 2 * np.arctan2(x*np.sin(angle/2), np.cos(angle/2))
        attitude = np.pi/2
        bank = 0
        return heading, attitude, bank
    
    elif ((x*y*t + z*s) < -0.998):
        
        heading = -2 * np.arctan2(x*np.sin(angle/2), np.cos(angle/2))
        attitude = -np.pi/2
        bank = 0
        return heading, attitude, bank
    
    heading = np.arctan2(y * s- x * z * t , 1 - (y*y+ z*z ) * t)
    attitude = np.arcsin(x * y * t + z * s)
    bank = np.arctan2(x * s - y * z * t , 1 - (x*x + z*z) * t)
    
    return bank, heading, attitude


def getRotation(v1):
    if np.all(v1==[0., 0., 1.]): v1 = [0, 0.000001, 0.999999]
    vec_x = [1.,0.,0.] 
    vec_y = [0.,1.,0.]
    vec_z = [0.,0.,1.] #>>>>>>>>>>>>>>>>>>>>> change these to -1 to flip again
    
    vec1 = v1 / np.linalg.norm(v1)

    #vector_x = np.cross(vec1, vec_x)/np.linalg.norm(np.cross(vec1, vec_x))
    angle_x = (math.acos(np.dot(vec1, vec_x)))

    #vector_y = np.cross(vec1, vec_y)/np.linalg.norm(np.cross(vec1, vec_y))
    angle_y = (math.acos(np.dot(vec1, vec_y)))

    vector_z = np.cross(vec1, vec_z)/np.linalg.norm(np.cross(vec1, vec_z))
    angle_z = -(math.acos(np.dot(vec1, vec_z)))  #<<<Changed to adapt

    #Rotation = filtered_Pt_Cloud.get_rotation_matrix_from_axis_angle(angle*vector) #alternative Open3D lib.
    Rotation = R.from_rotvec(angle_z*vector_z)
    
    return Rotation, angle_x, angle_y, angle_z


def Bounds_gen(minB, maxB, spacing):
    ctr = 0
    bounds = np.array([[0.,0.]])
    CurrB = minB

    while CurrB < maxB-0.5*spacing: #from left most to right most

        LowerB = minB + ctr * spacing #if we shift the X or Y coordinates in multiples of spacing, we should get different lines.
        if ctr == 0:
            CurrB = LowerB + spacing/2 #lower + spacing/2 only for first condition
        else:
            CurrB = LowerB + spacing #lower + spacing
        bounds = np.append(bounds,[[LowerB,CurrB]], axis=0)
        #print("CurrB:",CurrB,"MaxB:",maxB,"diff:",maxB-CurrB)
        ctr+= 1
    bounds = np.delete(bounds, 0, axis=0)
    return bounds

def cropped_PC(original_PC, spacing, X=0, idx = 0, centered = True):
    PC_BBOX = original_PC.get_axis_aligned_bounding_box()
    minB_X = PC_BBOX.min_bound[0]
    maxB_X = PC_BBOX.max_bound[0]
    minB_Y = PC_BBOX.min_bound[1]
    maxB_Y = PC_BBOX.max_bound[1]
    minB_Z = PC_BBOX.min_bound[2]
    maxB_Z = PC_BBOX.max_bound[2]
    nor = np.array(original_PC.normals)
    pts = np.array(original_PC.points)

    Xs,Ys = get_XY_angles_from_PC(nor)
    X_dev = np.std(Xs)
    Y_dev = np.std(Ys)
    print("X Standard Dev:", X_dev, "    Y Standard Dev:", Y_dev)
    
    if(X > 1):
        if(X==2): #Manual override at 2
            X = 0  
            print("Manual Override! taking curve around World Y-axis","X=", X)
        elif(X==3):  #Manual override at 3
            X = 1
            print("Manual Override! taking curve around World X-axis","X=", X)
        else: X = 1
    else:
        
        if (X_dev <= Y_dev): 
            X = 1
            print("Object's curve around World X-axis","X=", X)
        elif (X_dev > Y_dev): 
            X = 0
            print("Object's curve around World Y-axis","X=", X)

    if X == 1: #X sided sweep crop
        bounds = Bounds_gen(minB_X, maxB_X, spacing)
        print("Total bounds:",len(bounds))
        if centered == True:
            idx = int(np.floor(len(bounds)/2))
        else:
            if idx > len(bounds)-1:
                print("Warning! Index value out of bounds, setting to max value")
                idx = -1
        bbox = o3d.geometry.AxisAlignedBoundingBox(min_bound=(bounds[idx][0], minB_Y, minB_Z), max_bound=(bounds[idx][1], maxB_Y, maxB_Z))

        result = original_PC.crop(bbox)
        res_pts = np.array(result.points)
        res_nor = np.array(result.normals)

        ind = np.argsort(res_pts[:, 1]) #Sort Y coordinates from lowest to highest, X values are almost constant
        res_pts = res_pts[ind]  #no longer required to change values to negative 
        res_nor = res_nor[ind]

    elif X == 0: #Y sided sweep crop
        bounds = Bounds_gen(minB_Y, maxB_Y, spacing)
        print("Total bounds:",len(bounds))
        if centered == True:
            idx = int(np.floor(len(bounds)/2))
        else:
            if idx > len(bounds)-1:
                print("Warning! Index value out of bounds, setting to max value")
                idx = -1
        bbox = o3d.geometry.AxisAlignedBoundingBox(min_bound=(minB_X, bounds[idx][0], minB_Z), max_bound=(maxB_X, bounds[idx][1], maxB_Z)) 
        print(bbox)

        result = original_PC.crop(bbox)
        res_pts = np.array(result.points)
        res_nor = np.array(result.normals)

        ind = np.argsort(res_pts[:, 0]) #Sort X coordinates from lowest to highest, Y values are almost constant
        res_pts = res_pts[ind]
        #res_pts[:,2] *=-1  #Change z values to negative
        res_nor = res_nor[ind]

    sorted_pointcloud = o3d.geometry.PointCloud()
    sorted_pointcloud.points = o3d.utility.Vector3dVector(res_pts)
    sorted_pointcloud.normals = o3d.utility.Vector3dVector(res_nor)
    return sorted_pointcloud


def generate_coordinates(point_cloud):
    Coordinates = np.array([[0.,0.,0.,0.,0.,0.,0.]])
    world_Coordinates = np.array([[0.,0.,0.,0.,0.,0.,0.]])
    point_cloud_pts = np.array(point_cloud.points)
    point_cloud_nor = np.array(point_cloud.normals)
    
    for index in range (len(point_cloud_pts)):

        rotat, angle_x, angle_y, angle_z = getRotation(-point_cloud_nor[index])  #<<<changed here pts -ve

        a1 = rotat.as_matrix()[0][0]
        a2 = rotat.as_matrix()[0][1]
        a3 = rotat.as_matrix()[0][2]
        b1 = rotat.as_matrix()[1][0]
        b2 = rotat.as_matrix()[1][1]
        b3 = rotat.as_matrix()[1][2]
        c1 = rotat.as_matrix()[2][0]
        c2 = rotat.as_matrix()[2][1]
        c3 = rotat.as_matrix()[2][2]

        KDL_original_plane_frame = PyKDL.Frame(PyKDL.Rotation(a1,a2,a3, b1,b2,b3, c1,c2,c3),
                                               PyKDL.Vector(point_cloud_pts[index][0],
                                                            point_cloud_pts[index][1],
                                                             point_cloud_pts[index][2]))

        KDL_flip_frame = PyKDL.Frame(PyKDL.Rotation.RPY(0, np.pi, 0), PyKDL.Vector(0,0,0)) #mirror plane frame to match camera frame
        KDL_final_frame = KDL_original_plane_frame * KDL_flip_frame
        KDL_trans = KDL_final_frame.p
        KDL_ROT_quat = KDL_final_frame.M.GetQuaternion() 

        Coordinates = np.append(Coordinates,[[KDL_trans[0] ,KDL_trans[1],KDL_trans[2] ,KDL_ROT_quat[0] 
                                     ,KDL_ROT_quat[1] ,KDL_ROT_quat[2],KDL_ROT_quat[3]]],axis=0)

        Publish_coordinates([[KDL_trans[0] ,KDL_trans[1],KDL_trans[2] ,KDL_ROT_quat[0] 
                            ,KDL_ROT_quat[1] ,KDL_ROT_quat[2],KDL_ROT_quat[3]]], 
                            'camera_depth_optical_frame', 'plane', static = True)

        transform_plane = fetch_transform(tfbuffer,'world', 'plane_static_0',quat=1)

        world_Coordinates = np.append(world_Coordinates,[[transform_plane[0] ,transform_plane[1]
                                            ,transform_plane[2] ,transform_plane[3] ,transform_plane[4] 
                                             ,transform_plane[5] ,transform_plane[6]]],axis=0)

    Coordinates = np.delete(Coordinates, 0, axis=0)
    world_Coordinates2 = np.delete(world_Coordinates, 0, axis=0)
    return Coordinates, world_Coordinates2

def Publish_coordinates(Coordinates, parent_name, child_name, static = False): #Coordinates expects [[0,0,0,0,0,0,0],[1,1,1,1,1,1,1]] format

    for index in range (len(Coordinates)):
        
        if static==True:
            static_t.header.stamp = rospy.Time.now()
            static_t.header.frame_id = parent_name #"camera_depth_optical_frame"
            static_t.child_frame_id = child_name+"_static_"+str(index)
            static_t.transform.translation.x = Coordinates[index][0]
            static_t.transform.translation.y = Coordinates[index][1]
            static_t.transform.translation.z = Coordinates[index][2]

            #r_quat = tf.transformations.quaternion_from_euler(Roll,Pitch,Yaw)
            static_t.transform.rotation.x = Coordinates[index][3]
            static_t.transform.rotation.y = Coordinates[index][4]
            static_t.transform.rotation.z = Coordinates[index][5]
            static_t.transform.rotation.w = Coordinates[index][6]
            static_br.sendTransform(static_t)
        else:
            t.header.frame_id = parent_name #"camera_depth_optical_frame"
            t.child_frame_id = child_name+"_"+str(index)

            t.header.stamp = rospy.Time.now()
            t.transform.translation.x = Coordinates[index][0]
            t.transform.translation.y = Coordinates[index][1]
            t.transform.translation.z = Coordinates[index][2] 

            #r_quat = tf.transformations.quaternion_from_euler(Roll,Pitch,Yaw)
            t.transform.rotation.x = Coordinates[index][3]
            t.transform.rotation.y = Coordinates[index][4]
            t.transform.rotation.z = Coordinates[index][5]
            t.transform.rotation.w = Coordinates[index][6]
            br.sendTransform(t)


def Cluster_Point_Cloud(original_PC, eps=0.02, min_points=10):

    labels = np.array(original_PC.cluster_dbscan(eps=eps, min_points=min_points))
    uniques = np.unique(labels)
    clouds = np.array(original_PC)
    
    if ((uniques[0] == -1) and (len(uniques) == 1)):
        clouds = np.append(clouds,[original_PC])

    else:
        for i in range (len(uniques)):

            if uniques[i] > -1:
                idx = np.where(labels==uniques[i])[0]
                cluster_pcd = original_PC.select_by_index(idx, invert=False)
                clouds = np.append(clouds,[cluster_pcd])

    clouds = np.delete(clouds, 0)
    print("Number of clusters:", len(clouds))
    return clouds


def go_to_coord_goal(move_group,coord):
    pose_goal = geometry_msgs.msg.Pose()
    pose_goal.position.x = coord[0]
    pose_goal.position.y = coord[1]
    pose_goal.position.z = coord[2]
    pose_goal.orientation.x = coord[3]
    pose_goal.orientation.y = coord[4]
    pose_goal.orientation.z = coord[5]
    pose_goal.orientation.w = coord[6]

    move_group.set_pose_target(pose_goal)
    
    success = move_group.go(wait=True)
    
    move_group.stop()
    move_group.clear_pose_targets()
    current_pose = move_group.get_current_pose().pose

def array_to_data(array):
    im = PIL_img.fromarray(array)
    output_buffer = BytesIO()
    im.save(output_buffer, format="PNG")
    data = output_buffer.getvalue()
    return data

def fetch_cloud_image(pointCloud, RX=0, RY=0, RZ=0):
    mesh = o3d.geometry.TriangleMesh.create_coordinate_frame(size=0.1,origin=[0, 0, 0])
    mesh_pc= mesh.sample_points_uniformly(number_of_points=1000000, use_triangle_normal=False)
    tmp_cloud = copy.deepcopy(pointCloud)  #To avoid overwriting the original point cloud
    
    tmp_cloud+=mesh_pc
    tmp_Rot = pointCloud.get_rotation_matrix_from_xyz((np.radians(RX), np.radians(RY), np.radians((RZ))))
    
    tmp_cloud.rotate(tmp_Rot, center=(0, 0, 0))
    
    vis = o3d.visualization.Visualizer() 
    vis.create_window(visible=False, width=640, height=480) 
    vis.add_geometry(tmp_cloud) 
    vis.poll_events() 
    vis.update_renderer() 
    color = vis.capture_screen_float_buffer(True) 
    #time.sleep(5)
    vis.destroy_window() 
    #color = np.asarray(color)
    color = (255.0 * np.asarray(color)).astype(np.uint8)
    color = array_to_data(color) #Format according to the GUI requirements
    return color

def Cluster_selection_gui(clouds):
    dat=[]  #to hold cluster's images
    pcd_combined = o3d.geometry.PointCloud()
    
    for current_cloud in clouds:
            color = fetch_cloud_image(current_cloud, RX=120,RZ=180)  #different point clouds to be viewed, set view by RX,RY,RZ
            dat.append(color)
    result = subprocess.run([sys.executable,  "Cluster_selection_gui.py"],capture_output=True,text=True,check=True,shell=False, input=repr(dat))   
    OP = literal_eval(result.stdout)
    selected_PC = OP[0]
    selected_mode = OP[1]
    
    for k in selected_PC:
        pcd_combined += clouds[k]  #combine selected pointclouds 
    return pcd_combined, selected_mode

#def Convert_message(string):
#    return [int(s) for s in re.findall('[0-9]', string)]

def get_XY_angles_from_PC(tst_downpcd_nor):
    Xs = []
    Ys = []
    vec_x = [1.,0.,0.] 
    vec_y = [0.,1.,0.]
    
    for normal_vec in tst_downpcd_nor:
        if np.all(normal_vec==[0., 0., 1.]): normal_vec = [0, 0.000001, 0.999999]
        vec1 = normal_vec / np.linalg.norm(normal_vec)
        angle_x = np.round(np.degrees((math.acos(np.dot(vec1, vec_x)))))
        angle_y = np.round(np.degrees((math.acos(np.dot(vec1, vec_y)))))
        Xs.append(angle_x)
        Ys.append(angle_y)
    Xs = np.array(Xs)
    Ys = np.array(Ys)
    return Xs, Ys


print("Loaded!!")

Loaded!!


In [20]:
#move_group.get_current_pose()
initial_coordinates = [0.408,0.,0.834,0.9238709648606045,-0.3826807475252162,-0.003837162229149499,0.001778186465088758]

initial_coordinates_vertical = [0.28701854121331966,-0.03260223409115588,0.44804421741139216,
                       -0.6945874945768339,0.29634459105695476,-0.6174160637305418,0.2202850425613284]

move_group.get_current_pose()

go_to_coord_goal(move_group, initial_coordinates) #pass values to function to make robot move in cartesian space. 
print("Moving to initial position")

Moving to initial position


In [21]:
spacing = 0.01 #spacing between each point in point cloud in meters (use this to approximate the shape of surface)
offset_y = 0.13 #distance to crop in world's X axis away from robot, to avoid robot's shadow appearing in PC.
offset_z = 0.9
mesh = o3d.geometry.TriangleMesh.create_coordinate_frame(size=0.1,origin=[0, 0, 0])

point = (320, 240)


#_, depth_frame, color_frame = dc.get_frame()
color_frame, depth_frame = grab_frame()
print("color frame:",color_frame.shape, " Depth frame:",depth_frame.shape)
distance = depth_frame[point[1], point[0]]
print(distance)
print("Center of image:", distance, "mm")

pt_cloud = Generate_PointCloud(color_frame,depth_frame,from_depth=False)


downpcd = pt_cloud.voxel_down_sample(voxel_size=spacing)

print("Recompute the normal of the downsampled point cloud")
downpcd.estimate_normals(search_param=o3d.geometry.KDTreeSearchParamHybrid(radius=0.06, max_nn=30)) #radius in meters
print("Align normals towards camera")
downpcd.orient_normals_towards_camera_location(camera_location=[0, 0, 0])

#o3d.visualization.draw_geometries([downpcd,mesh], point_show_normal=True)

## Filter out hidden points
diameter = np.linalg.norm(np.asarray(downpcd.get_max_bound()) - np.asarray(downpcd.get_min_bound()))
cam = [0, 0, diameter]
radius = diameter * 1000
_, pt_map = downpcd.hidden_point_removal(cam, radius) #Get all points that are visible from given view point
downpcd = downpcd.select_by_index(pt_map)

o3d.visualization.draw_geometries([downpcd,mesh], point_show_normal=True)

#v2_camera = np.array(downpcd.normals)
v2_camera_pts = np.array(downpcd.points)


#idx = np.where(abs(v2_camera_pts[:,2]) < (distance/1000)+0.1)[0] #fetch all indexes of values less than distance of center of img 0.3 in column 3 of rec
idx = np.where(abs(v2_camera_pts[:,2]) < abs(np.min(v2_camera_pts[:,2]))-0.06)[0] #fetch all indexes of values less than distance of center of img 0.3 in column 3 of rec

#print(idx)
filtered_Pt_Cloud = downpcd.select_by_index(idx, invert=False)


#CROP TO FILTER OUT ROBOT'S SHADOW ADJUST OFFSET ACCORDINGLY
PC_BBOX = filtered_Pt_Cloud.get_axis_aligned_bounding_box()
print(PC_BBOX)
minB_X = PC_BBOX.min_bound[0]
maxB_X = PC_BBOX.max_bound[0]
minB_Y = PC_BBOX.min_bound[1]
maxB_Y = PC_BBOX.max_bound[1]
minB_Z = PC_BBOX.min_bound[2]
maxB_Z = PC_BBOX.max_bound[2]
bbox = o3d.geometry.AxisAlignedBoundingBox(min_bound=(minB_X, minB_Y, minB_Z+offset_z), max_bound=(maxB_X, maxB_Y-offset_y, maxB_Z)) 
filtered_Pt_Cloud = filtered_Pt_Cloud.crop(bbox)

Re = filtered_Pt_Cloud.get_rotation_matrix_from_xyz((np.pi, 0 , 0))
filtered_Pt_Cloud = filtered_Pt_Cloud.rotate(Re, center=(0,0,0))

print(filtered_Pt_Cloud)
o3d.visualization.draw_geometries([filtered_Pt_Cloud,mesh], point_show_normal=True)



nor = np.array(filtered_Pt_Cloud.normals)
pts = np.array(filtered_Pt_Cloud.points)

print()
distance,index = spatial.KDTree(pts).query( filtered_Pt_Cloud.get_center() ) #find coordinates 
                                                                             #that are closest to the center 

print(distance)
print(index)
print()
print("Center Coordinates(ground truth):",filtered_Pt_Cloud.get_center())
print("Points Coordinates(estimated center point):",pts[index])
print("Normal Coordinates(Normal of estimated center point):",nor[index])

color frame: (480, 640, 3)  Depth frame: (480, 640)
556
Center of image: 556 mm
Recompute the normal of the downsampled point cloud
Align normals towards camera
AxisAlignedBoundingBox: min: (-0.755789, -0.609844, -1.58657), max: (1.34372, 0.976633, -0.159617)
PointCloud with 546 points.

0.011475102871330555
471

Center Coordinates(ground truth): [-0.02632869 -0.04871436  0.54905188]
Points Coordinates(estimated center point): [-0.02742619 -0.04715836  0.53773586]
Normal Coordinates(Normal of estimated center point): [ 0.12867676  0.10779038 -0.9858111 ]


### Clustering of points in Point cloud
ref: http://www.open3d.org/docs/release/tutorial/geometry/pointcloud.html#DBSCAN-clustering

### Crop point cloud to get a points in a single line in X or Y axis, 
*Use idx to give offset else use centered to get the points from center of the object*

In [22]:
clouds = Cluster_Point_Cloud(filtered_Pt_Cloud, eps=0.05, min_points=10)

result,selected_mode = Cluster_selection_gui(clouds) #Will present a GUI to select all relevant pointclouds
#o3d.visualization.draw_geometries([result, mesh], point_show_normal=True)

result = cropped_PC(result, spacing, X=selected_mode, idx = 0, centered = True)
o3d.visualization.draw_geometries([result, mesh], point_show_normal=True)

Number of clusters: 1
X Standard Dev: 32.23175566252703     Y Standard Dev: 3.319452849135058
Object's curve around World Y-axis X= 0
Total bounds: 16
AxisAlignedBoundingBox: min: (-0.129168, -0.0541229, 0.532244), max: (0.0602247, -0.0441229, 0.604833)


### This will generate and boradcast all points in pointcloud to tf2 or tf2_static topic, visualize using Rviz

In [23]:
cam_coords, world_coords = generate_coordinates(result)
#cam_coords, world_coords = generate_coordinates(clouds[0])
print("Total coordinates:",len(world_coords))

Total coordinates: 30


In [26]:
#Publish_coordinates(coords,'camera_depth_optical_frame','plane' static = False)
for i in range (len(world_coords)):
    Publish_coordinates([world_coords[i]],'world','plane', static = True)
    time.sleep(0.1)


## Robot Motion

In [31]:
z_offset = 0.3

Publish_coordinates([world_coords[0]],'world','plane', static = True)
time.sleep(5)

for id_x in range(0,len(world_coords),3):  #Take every 4th point in coordinates
    transform_world_plane = world_coords[id_x]
    KDL_original_plane_frame = PyKDL.Frame(PyKDL.Rotation.Quaternion(transform_world_plane[3],
                                                                     transform_world_plane[4], 
                                                                     transform_world_plane[5], 
                                                                     transform_world_plane[6]),
                                           PyKDL.Vector(transform_world_plane[0], 
                                                        transform_world_plane[1], 
                                                        transform_world_plane[2]))

    trans_x,trans_y,trans_z = KDL_original_plane_frame * PyKDL.Vector(0, 0, z_offset) #Add offset
    KDL_original_plane_frame.p = PyKDL.Vector(trans_x,trans_y,trans_z) #update original plane frame to new location

    KDL_flip_frame = PyKDL.Frame(PyKDL.Rotation.RPY(np.pi, 0.,np.pi), PyKDL.Vector(0, 0, 0)) #mirror plane frame to match camera frame

    KDL_final_frame = KDL_original_plane_frame * KDL_flip_frame

    KDL_trans = KDL_final_frame.p
    KDL_ROT_quat = KDL_final_frame.M.GetQuaternion() 

    final_coordinates = [KDL_trans[0], KDL_trans[1], KDL_trans[2], KDL_ROT_quat[0], KDL_ROT_quat[1], 
                                                                   KDL_ROT_quat[2], KDL_ROT_quat[3]]

    print(final_coordinates)

    Publish_coordinates([final_coordinates], "world", 'Camera_Target', static = True)
    
    #Fetch transform between link8 and optical depth cam
    transform_link8_camera_depth = fetch_transform(tfbuffer,'camera_depth_optical_frame', 'panda_link8',quat=1)

    KDL_link8_cam_frame = PyKDL.Frame(PyKDL.Rotation.Quaternion(transform_link8_camera_depth[3],
                                                                transform_link8_camera_depth[4], 
                                                                transform_link8_camera_depth[5],
                                                                transform_link8_camera_depth[6]),
                                      PyKDL.Vector(transform_link8_camera_depth[0], 
                                                   transform_link8_camera_depth[1], 
                                                   transform_link8_camera_depth[2])) #link8-camera frame

    #Offset original frame by z offset, flip the frame to match camera, multiply with link8-camera frame to replicate
    #pose between camera and link8
    KDL_final_frame = KDL_original_plane_frame * KDL_flip_frame * KDL_link8_cam_frame 

    KDL_trans = KDL_final_frame.p
    KDL_ROT_quat = KDL_final_frame.M.GetQuaternion() 

    final_coordinates = [KDL_trans[0], KDL_trans[1], KDL_trans[2], KDL_ROT_quat[0], KDL_ROT_quat[1], 
                                                                   KDL_ROT_quat[2], KDL_ROT_quat[3]]
    Publish_coordinates([final_coordinates], "world", 'EEF_Target', static = True)

    ## Move Robot to TARGET
    go_to_coord_goal(move_group, final_coordinates) #pass values to function to make robot move in cartesian space. 
    print("Moving to Target")
    #time.sleep(0.2)

[0.5169460752600362, -0.2931378109959326, 0.513008685939273, -0.6681884199802429, 0.6700613747685901, 0.21741105176519843, 0.23932075551001514]
Moving to Target
[0.48275791842468135, 0.3029990461584236, 0.5424310002008139, 0.6723376049115928, -0.6706358336686491, 0.2510431018299105, 0.18758220769816047]
Moving to Target
[0.47886321568535833, 0.28336987425503724, 0.556188698573584, 0.6753621328780705, -0.6737497771799281, 0.2450104375878842, 0.17296564022190705]
Moving to Target
[0.4740081452637322, 0.2124820160832368, 0.597251793515541, 0.6911346647084695, -0.6900721842291394, 0.18922248798458868, 0.1016272888178102]
Moving to Target
[0.4770829722063061, 0.12467244219755635, 0.622697406949539, 0.7037995883473983, -0.7034794880516034, 0.09634621581564563, 0.02236416845516313]
Moving to Target
[0.48181057100605745, -0.03133488004538708, 0.6199474026018607, -0.7015602473449096, 0.7024256668774993, 0.04724339852003739, 0.11036060507865131]
Moving to Target
[0.48332572734072266, -0.07713577